In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import chain
sns.set()


def bar_annotation(ax, option='number'):
    '''
    This function is to add according number or percentage on every bar
    option: number or percentage, default is number
    ax: the figure object
    '''
    try:
        if option == 'number':
            for i in ax.patches:
                ax.text(i.get_x(), i.get_height()+.5, i.get_height(),
                         fontsize=15, color='dimgrey')
        if option == 'percentage':
            totals = []
            for i in ax.patches:
                totals.append(i.get_height())
                total = sum(totals)
            for i in ax.patches:
                ax.text(i.get_x(), i.get_height()+.5,
                        str(round((i.get_height()/total)*100, 2))+'%',
                        fontsize=15, color='dimgrey')
    except:
        return('inputs are wrong')


def total_seconds(time):
    '''
    Convert the resolution of time to hours
    '''
    return time.total_seconds()/3600

root_dir = 'C:/Users/gzhu1/Desktop/SAH_data'
patient = pd.read_csv(
        root_dir+"/patient_encounter/Pat_file_ENC.csv")
patient['PATIENT_SK'].nunique()                      # 4810
SAH_ENC_screened_24h_rule = pd.read_csv(
  root_dir+"/patient_encounter/SAH_ENC_screened_24h_rule.csv")
SAH_ENC_screened_24h_rule['PATIENT_SK'].nunique()    # 4810
SAH_ENC_screened_24h_rule['ENCOUNTER_ID'].nunique()  # 4823
SAH_ENC_screened_24h_rule.shape
# Get a new patient level table by adding information such as encounter ID
pat_enc_final = pd.merge(patient, SAH_ENC_screened_24h_rule, on='PATIENT_SK')
pat_enc_final.shape


# -----------------------------------------------------------------------------------
# Load the medication tables
# -----------------------------------------------------------------------------------

pd.options.display.max_rows = 50
pd.options.display.max_columns = 10
enc_med_cl2 = pd.read_csv(root_dir+'/medication/sah_pts_enc_med_sl_ad_cl2_gz.tsv',
                          delimiter='\t')
enc_med_cl3 = pd.read_csv(root_dir+'/medication/sah_pts_all_enc_med_cl3_v2.tsv',
                          delimiter='\t')
# In order to get the dose quantity,
# we need to merge some columns between enc_med_cl2 and enc_med_cl3
enc_med_cl2 = pd.merge(enc_med_cl2, enc_med_cl3[['INDEX_ID', 'DOSE_QUANTITY',
                       'ORDER_NO', 'TOTAL_DISPENSED_DOSES', 'CREDIT_QUANTITY',
                       'CHARGE_QUANTITY' ,'INFUSION_TIME',
                       'DOSE_FORM_DESCRIPTION', 'ROUTE_DESCRIPTION',
                       'INITIAL_DOSE_QUANTITY']],
                        on='INDEX_ID')

enc_med_cl2['MED_STARTED_DT_TM'] = pd.to_datetime(
        enc_med_cl2['MED_STARTED_DT_TM'])
start_tm_na_ind = (enc_med_cl2['MED_STARTED_DT_TM'].apply(str)
                   == '01-JAN-1000 00:00:00')
stop_tm_na_ind = (enc_med_cl2['MED_STOPPED_DT_TM'].apply(str)
                  == '01-JAN-1000 00:00:00')
enc_med_cl2['MED_STOPPED_DT_TM'][stop_tm_na_ind] = np.nan
enc_med_cl2['MED_STOPPED_DT_TM'] = pd.to_datetime(
        enc_med_cl2['MED_STOPPED_DT_TM'])
enc_med_cl2['stop_start_gap'] = enc_med_cl2['MED_STOPPED_DT_TM'] -\
                 enc_med_cl2['MED_STARTED_DT_TM']
enc_med_cl2['stop_start_gap'] = enc_med_cl2['stop_start_gap'].\
                                apply(total_seconds)
discon_tm_na_ind = (enc_med_cl2['MED_DISCONTINUED_DT_TM'].apply(str)
                  == '01-JAN-1000 00:00:00')
sum(discon_tm_na_ind)
enc_med_cl2['MED_DISCONTINUED_DT_TM'][discon_tm_na_ind] = np.nan
enc_med_cl2['MED_DISCONTINUED_DT_TM'] = pd.to_datetime(
        enc_med_cl2['MED_DISCONTINUED_DT_TM'])
# Load NDC codes
vasopressor_ndc_codes = pd.read_csv(
        root_dir+'/medication/list of vasopressors with ndc codes.tsv',
        delimiter='\t')
vasopressor_ndc_codes = vasopressor_ndc_codes[[
        'MEDICATION_ID', 'NDC_CODE']].drop_duplicates()
vasopressor_ndc_codes['MEDICATION_ID'].nunique()
vasopressor_ndc_codes['NDC_CODE'].nunique()
enc_med_cl2 = pd.merge(vasopressor_ndc_codes, enc_med_cl2, on='MEDICATION_ID',
                       how='inner')



C:\Users\gzhu1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\gzhu1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
ndc_517180525 = enc_med_cl2.query('NDC_CODE=="517180525"')
                                  
ndc_517180525[['NDC_CODE',
                           "TOTAL_DISPENSED_DOSES",'CONSUMED_QUANTITY',
                           'CHARGE_QUANTITY','CREDIT_QUANTITY','DOSE_QUANTITY',
                           'ORDER_STRENGTH', 'ORDER_VOLUME',
                           'TOTAL_VOLUME']]

,NDC_CODE,TOTAL_DISPENSED_DOSES,CONSUMED_QUANTITY,CHARGE_QUANTITY,CREDIT_QUANTITY,DOSE_QUANTITY,ORDER_STRENGTH,ORDER_VOLUME,TOTAL_VOLUME
23,517180525,42,42,42,0,1,NaN,NaN,0
24,517180525,8,8,8,0,1,48.0,30.0,0
25,517180525,28,280,420,140,1,NaN,230.0,250
26,517180525,1,1,1,0,1,200.0,5.0,5
27,517180525,1,1,1,0,1,200.0,5.0,5
28,517180525,1,1,1,0,1,200.0,5.0,5
29,517180525,1,1,1,0,1,200.0,5.0,5
30,517180525,1,1,1,0,1,200.0,5.0,5
31,517180525,1,1,1,0,1,200.0,5.0,5
32,517180525,1,1,1,0,1,200.0,5.0,5
